In [6]:
import pandas as pd
import random
import json
import re
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import BaggingClassifier
from sklearn.multiclass import OneVsRestClassifier
import joblib
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from tqdm import tqdm
import string
import scipy
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
stop_words = stopwords.words('english')
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader
from datasets import load_dataset
import pandas as pd
from sklearn.metrics import f1_score
from IPython.display import HTML, display
from tqdm import tqdm
import spacy
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordTokenizer
stop_words = stopwords.words('english')
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
tokenizer = TreebankWordTokenizer()

[nltk_data] Downloading package stopwords to /home/jimbo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jimbo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jimbo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jimbo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jimbo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jimbo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [11]:
def text_to_doccano(sentences):
    """
    :text (str): source text
    Returns (list (dict)): deccano format json
    """
    djson = list()
    #doc = nlp(text)
    for sentence in tqdm(sentences):
        labels = list()
        for e in sentence.ents:
            labels.append([e.start_char, e.end_char, e.label_])
        djson.append({'text': sentence.text, "labels": labels})
    return djson

In [33]:
clutter_model = spacy.load('../NER_models/declutter-1-3500-corrected-full-dataset/model-best')
training_data = pd.read_csv('../datasets/All-Copyrights-English-Cleaned.csv')
training_data_entities = [clutter_model(sentence) for sentence in tqdm(training_data['copyright'])]

100%|██████████| 15405/15405 [00:34<00:00, 452.43it/s]


In [34]:
doccano_json = text_to_doccano(training_data_entities)
with open('All-Copyrights-English-Cleaned.jsonl', 'w') as f:
    for item in doccano_json:
        f.write(json.dumps(item) + '\n')

100%|██████████| 15405/15405 [00:00<00:00, 101612.50it/s]


In [35]:
# Convert JSONL to spacy compatible format
import json
import spacy
from spacy.tokens import Doc, DocBin
from spacy.training import Example
from spacy.util import minibatch

def load_data(file_path):
    """
    Load JSONL data from a file and convert it to spaCy's training format.
    :param file_path: The path of the JSONL file.
    :return: A list of training examples.
    """
    data = []
    with open(file_path, 'r') as f:
        for line in f:
            item = json.loads(line)
            text = item['text']
            entities = [(e[0], e[1], e[2]) for e in item['label']]
            data.append((text, {'entities': entities}))
    return data

def save_data(data, file_path):
    """
    Save data in spaCy's binary format.
    :param data: The training data.
    :param file_path: The path of the output file.
    """
    nlp = spacy.blank('en')
    doc_bin = DocBin()
    for text, annotations in data:
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)
        doc_bin.add(example.reference)
    doc_bin.to_disk(file_path)

In [36]:
data = load_data('../NER_datasets/default/declutter-data/4000-cleaned-up-data.jsonl')
save_data(data, '../NER_datasets/spacy_compatible/declutter-data/4000-cleaned-up-data.spacy')

/home/jimbo/pythondeps/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "(C) 1988 Landon Curt Noll & Ronald S. Karr"</s> (o..." with entities "[(0, 42, 'Copyright')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/home/jimbo/pythondeps/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "(C) 1994-2008 Christos Zoulas <christos@zoulas.com..." with entities "[(0, 51, 'Copyright')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/home/jimbo/pythondeps/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "(C) 1995-2017 Jean-loup Gailly & Mark Adler\0" VAL..." with entities "[(0, 43, 'Copyright')]

In [39]:
# split the 600 documents into train and test - 80, 20 split.
doc_bin = spacy.tokens.DocBin().from_disk('../NER_datasets/spacy_compatible/declutter-data/4000-cleaned-up-data.spacy')
nlp = spacy.load("en_core_web_sm")
docs = list(doc_bin.get_docs(nlp.vocab))
train_docs, test_docs = train_test_split(docs, test_size=0.2, random_state=42, shuffle=True)
train_doc_bin = spacy.tokens.DocBin(docs=train_docs)
test_doc_bin = spacy.tokens.DocBin(docs=test_docs)
train_doc_bin.to_disk('../NER_datasets/spacy_compatible/declutter-data/4000-cleaned-up-data-train.spacy')
test_doc_bin.to_disk('../NER_datasets/spacy_compatible/declutter-data/4000-cleaned-up-data-valid.spacy')

/home/jimbo/pythondeps/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.5.0) was trained with spaCy v3.5.0 and may not be 100% compatible with the current version (3.6.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [42]:
!python -m spacy debug data '../NER_configs/declutter-4000-cleaned-up-data.cfg'


============================ Data file validation ============================
✔ Pipeline can be initialized with data
✔ Corpus is loadable

=============================== Training stats ===============================
Language: en
Training pipeline: tok2vec, ner
3200 training docs
800 evaluation docs
✔ No overlap between training and evaluation data

============================== Vocab & Vectors ==============================
ℹ 48188 total word(s) in the data (3902 unique)
ℹ No word vectors present in the package

========================== Named Entity Recognition ==========================
ℹ 1 label(s)
0 missing value(s) (tokens with '-' label)
✘ 2 invalid whitespace entity spans
✔ Good amount of examples for all labels
✔ Examples without occurrences available for all labels
✔ No entities crossing sentence boundaries
Entity spans consisting of or starting/ending with whitespace characters are
considered invalid.

================================== Summary ========================

In [43]:
!python -m spacy train '../NER_configs/declutter-4000-cleaned-up-data.cfg' --output '../NER_models/declutter-4000-cleaned-up-data'

✔ Created output directory:
../NER_models/declutter-4000-cleaned-up-data
ℹ Saving to output directory:
../NER_models/declutter-4000-cleaned-up-data
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     75.83   71.40   70.99   71.82    0.71
  0     200         91.71   1136.82   88.14   88.48   87.79    0.88
  0     400        100.82    337.41   84.29   84.96   83.64    0.84
  1     600        148.45    255.64   93.47   93.96   92.99    0.93
  2     800        300.52    303.75   92.85   92.97   92.73    0.93
  3    1000        273.61    248.66   92.60   92.60   92.60    0.93
  4    1200        412.18    286.56   93.68   93.99   93

In [44]:
nlp = spacy.load("../NER_models/declutter-4000-cleaned-up-data/model-best")
decluttered_data = [nlp(sentence) for sentence in tqdm(training_data['copyright'])]

100%|██████████| 15405/15405 [00:35<00:00, 432.30it/s]


In [46]:
spacy.displacy.serve(decluttered_data[4500:5000], style="ent", auto_select_port=True)

/home/jimbo/pythondeps/spacy/displacy/__init__.py:106: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  warnings.warn(Warnings.W011)



Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


In [48]:
spacy.displacy.serve(decluttered_data[9500:10000], style="ent", auto_select_port=True)


Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


In [55]:
spacy.displacy.serve(ansible_decluttered, style="ent", auto_select_port=True)

/home/jimbo/pythondeps/spacy/displacy/__init__.py:106: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  warnings.warn(Warnings.W011)
/home/jimbo/pythondeps/spacy/displacy/__init__.py:213: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)



Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


In [73]:
ansible = pd.read_csv('ansible-remove-false-positive.csv')
ansible.drop(ansible.columns.difference(['copyright']), axis=1, inplace=True)
ansible.drop_duplicates(inplace=True)
ansible_decluttered = [nlp(sentence) for sentence in tqdm(ansible['copyright'])]
spacy.displacy.serve(ansible_decluttered, style="ent", auto_select_port=True)

100%|██████████| 434/434 [00:01<00:00, 410.61it/s]
/home/jimbo/pythondeps/spacy/displacy/__init__.py:106: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  warnings.warn(Warnings.W011)
/home/jimbo/pythondeps/spacy/displacy/__init__.py:213: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)



Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


In [71]:
cassandra = pd.read_csv('cassandra-remove-false-positive.csv')
cassandra.drop(cassandra.columns.difference(['copyright']), axis=1, inplace=True)
cassandra.drop_duplicates(inplace=True)
cassandra_decluttered = [nlp(sentence) for sentence in tqdm(cassandra['copyright'])]
spacy.displacy.serve(cassandra_decluttered, style="ent", auto_select_port=True)

100%|██████████| 40/40 [00:00<00:00, 387.30it/s]
/home/jimbo/pythondeps/spacy/displacy/__init__.py:106: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  warnings.warn(Warnings.W011)



Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


In [69]:
vscode = pd.read_csv('vscode-remove-false-positive.csv')
vscode = vscode.drop_duplicates()
vscode.drop(vscode.columns.difference(['copyright']), axis=1, inplace=True)
vscode.drop_duplicates(inplace=True)
vscode_decluttered = [nlp(sentence) for sentence in tqdm(vscode['copyright'])]
spacy.displacy.serve(vscode_decluttered, style="ent", auto_select_port=True)

100%|██████████| 56/56 [00:00<00:00, 358.82it/s]
/home/jimbo/pythondeps/spacy/displacy/__init__.py:106: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  warnings.warn(Warnings.W011)
/home/jimbo/pythondeps/spacy/displacy/__init__.py:213: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)



Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


In [74]:
nlp_old = spacy.load("../NER_models/declutter-1-3500-corrected-full-dataset/model-best")
ansible = pd.read_csv('ansible-remove-false-positive.csv')
ansible.drop(ansible.columns.difference(['copyright']), axis=1, inplace=True)
ansible.drop_duplicates(inplace=True)
ansible_decluttered = [nlp_old(sentence) for sentence in tqdm(ansible['copyright'])]
spacy.displacy.serve(ansible_decluttered, style="ent", auto_select_port=True)

100%|██████████| 434/434 [00:01<00:00, 407.14it/s]
/home/jimbo/pythondeps/spacy/displacy/__init__.py:106: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  warnings.warn(Warnings.W011)



Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.
